In [2]:
# import os

# together_api_key = os.environ.get("TOGETHER_API_KEY")
# google_api_key = os.environ.get("GOOGLE_API_KEY")

# print(together_api_key)
# print(google_api_key)


In [3]:
# ohter reranking model: cross-encoder/ms-marco-MiniLM-L-6-v2

In [4]:
#!pip install python-dotenv sentence-transformers langchain langchain-google-genai

In [5]:
sample_text="""Question 1 Start
Question:  Analyze the common misconception that software engineering leads to excessive documentation and slows down development.  Using the provided text, explain why this is a myth and discuss the actual benefits of a well-defined software engineering process on project timelines and product quality.  Relate this to the concept of rework and its impact on project delivery.

Rubric:
- [Rubric 1 Start]Criterion 1: Identification and Explanation of the Myth - (2 marks) Accurately identifies the myth and provides a clear explanation of its misconception.
- [Rubric 2 Start]Criterion 2: Discussion of Benefits - (3 marks)  Clearly explains the benefits of a well-defined software engineering process on project timelines and product quality, referencing reduced rework.
- [Rubric 3 Start]Criterion 3: Connection between Rework and Delivery - (2 marks)  Effectively connects reduced rework to faster delivery times and provides a logical explanation.
- [Rubric 4 Start]Criterion 4:  Overall Clarity and Coherence - (3 marks) Presents a well-structured and coherent argument with clear and concise language.
Question 1 End


Question 2 Start
Question:  The text mentions that software is "engineered, not manufactured."  Explain the implications of this statement in the context of software development.  Compare and contrast the software engineering process with a traditional manufacturing process, highlighting at least three key differences.

Rubric:
- [Rubric 1 Start]Criterion 1: Explanation of the Core Difference - (2 marks)  Clearly explains the fundamental difference between engineering and manufacturing in the context of software.
- [Rubric 2 Start]Criterion 2: Comparison of Processes - (4 marks)  Provides a detailed comparison of the software engineering process and a traditional manufacturing process, highlighting at least three key differences.
- [Rubric 3 Start]Criterion 3:  Supporting Examples - (2 marks)  Supports the comparison with relevant and concrete examples.
- [Rubric 4 Start]Criterion 4:  Clarity and Organization - (2 marks) Presents a well-organized and clearly written response.
Question 2 End


Question 3 Start
Question:  Based on the provided text, describe the essential components of a comprehensive software engineering process.  Explain how these components contribute to the overall success of a software project, focusing on the management of change and the maintenance of quality.

Rubric:
- [Rubric 1 Start]Criterion 1: Identification of Key Components - (3 marks) Accurately identifies and describes the essential components of a software engineering process.
- [Rubric 2 Start]Criterion 2: Explanation of Contribution to Success - (3 marks)  Clearly explains how each component contributes to project success.
- [Rubric 3 Start]Criterion 3:  Focus on Change and Quality - (2 marks)  Specifically addresses the management of change and the maintenance of quality within the process.
- [Rubric 4 Start]Criterion 4:  Logical Structure and Clarity - (2 marks)  Presents a well-structured and clearly written response.
Question 3 End


Question 4 Start
Question: Evaluate the statement: "Whenever you think, we don’t have time for software engineering, ask yourself, ‘Will we have time to do it over again?’”  Discuss the long-term implications of neglecting a robust software engineering process, considering factors such as cost, time, and quality.

Rubric:
- [Rubric 1 Start]Criterion 1: Evaluation of the Statement - (2 marks)  Provides a thoughtful evaluation of the statement, considering its validity and implications.
- [Rubric 2 Start]Criterion 2: Discussion of Long-Term Implications - (4 marks)  Discusses the long-term implications of neglecting a robust software engineering process, considering cost, time, and quality.
- [Rubric 3 Start]Criterion 3:  Use of Supporting Evidence - (2 marks)  Supports the discussion with relevant examples or evidence.
- [Rubric 4 Start]Criterion 4:  Clarity and Persuasiveness - (2 marks)  Presents a clear, concise, and persuasive argument.
Question 4 End


Question 5 Start
Question: Synthesize the information provided to create a concise definition of software engineering.  Your definition should encompass the key aspects of the process, its goals, and its relationship to software development as a whole.

Rubric:
- [Rubric 1 Start]Criterion 1: Accuracy and Completeness - (3 marks)  Provides an accurate and comprehensive definition that captures the essence of software engineering.
- [Rubric 2 Start]Criterion 2:  Inclusion of Key Aspects - (3 marks)  Includes key aspects such as process, goals, and relationship to software development.
- [Rubric 3 Start]Criterion 3:  Clarity and Conciseness - (2 marks)  Presents a clear, concise, and well-written definition.
- [Rubric 4 Start]Criterion 4:  Use of Appropriate Terminology - (2 marks) Uses appropriate and precise terminology related to software engineering.
Question 5 End"""

In [6]:
import re
import json

def parse_questions(text: str, total_questions: int, path: str = None):
    all_questions = []

    for j in range(1, total_questions + 1):
        # Match the full block for the question
        block_pattern = re.compile(
            rf"Question {j} Start\s*(.*?)\s*Question {j} End", re.DOTALL
        )
        block_match = block_pattern.search(text)

        if not block_match:
            continue  # Skip if no match

        block = block_match.group(1)
        question_data = {"question_number": j}

        # Extract the question text
        question_match = re.search(r"Question:\s*(.*?)\s*Rubric:", block, re.DOTALL)
        question_data["question"] = question_match.group(1).strip() if question_match else "Not found"

        # Extract rubric entries
        rubric_entries = []
        # Updated regex to correctly capture criterion, description, and marks
        rubric_pattern = re.compile(
            r"^- \*\*(.*?):\*\* (.*?)\s*\((\d+)\s*marks\)$", re.MULTILINE
        )
        for rubric_match in rubric_pattern.finditer(block):
            rubric_entries.append({
                "criterion": rubric_match.group(1).strip(),
                "description": rubric_match.group(2).strip(),
                "marks": int(rubric_match.group(3))
            })

        question_data["rubric"] = rubric_entries
        all_questions.append(question_data)

    if path:
        with open(path, "w", encoding="utf-8") as f:
            json.dump(all_questions, f, indent=2)
        print(f"✅ JSON saved to {path}")
    
    return all_questions

In [7]:
#parse_questions(sample_text,5,r"C:\Users\dhili\Desktop\SRIP\week3\sample.json")

In [8]:
import os
from dotenv import load_dotenv
from sentence_transformers import CrossEncoder
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
load_dotenv() 
# Set your Together.ai API key
together_api_key = os.environ.get("TOGETHER_API_KEY")
googleapikey = os.environ.get("GOOGLE_API_KEY")# replace with actual key or load with dotenv
vector_db_path=r"week3\vector_db"
model="gemini-1.5-flash"
temp=0.3
# Instantiate your reranker onc

def get_embedding_function():
    return HuggingFaceEmbeddings(
        model_name="Qwen/Qwen3-Embedding-0.6B",  # Your model
          encode_kwargs={
        "batch_size": 4,  # or even 4 if you have limited VRAM
        "normalize_embeddings": True,
         #device="cpu"
    } # Optional: normalize for cosine similarity
    )
reranker = CrossEncoder(
    "BAAI/bge-reranker-v2-m3",
    max_length=512,
    #device="cpu"
)


# Main RAG function
def query_rag(query_text: str,no_of_questions:int,output_path:str,subject:str,chapter:int):
    # Load vector DB
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=vector_db_path, embedding_function=embedding_function)

    # Similarity search
    results = db.similarity_search_with_score(query_text, k=5)
        # 2. Prepare pairs for cross-encoder
    pairs = [(query_text, doc.page_content) for doc, _ in results]
    scores = reranker.predict(pairs)  # [score0, score1, …]

    # 3. Zip, sort by reranker score, and extract top docs
    reranked = sorted(
        zip(results, scores),
        key=lambda x: x[1],
        reverse=True
    )
    top_docs = [doc for (doc, _), _ in reranked]


    # 4. Build context from reranked docs
    annotated_chunks = []
    for doc in top_docs:
        file_name = doc.metadata.get("source", "unknown")
        page_num  = doc.metadata.get("page", "N/A")
        chunk_id  = doc.metadata.get("chunk_id", "N/A")
        annotated_chunks.append(
            f"[Source: {file_name}, Page {page_num}, Chunk {chunk_id}]\n"
            f"{doc.page_content}"
        )
    context_text = "\n\n---\n\n".join(annotated_chunks)

    # Prompt template
    prompt_template = ChatPromptTemplate.from_template("""# AI Academic Question Generator

You are an expert AI question generator that creates high-quality academic exam questions with detailed rubrics.

## Core Function
Generate analytical questions that test **higher-order thinking** (analysis, synthesis, evaluation) based on provided context and topic. Always include a comprehensive rubric with exactly 10 marks total.

## Input Requirements
- **Context**: Academic content/material provided
- **Topic**: Subject area or theme
- **Quantity**: Number of questions needed (1-20)
- **Level**: Academic level (undergraduate/graduate)

## Question Standards
- Require **multi-step reasoning** beyond memorization
- Use **clear, unambiguous language**
- Focus on **real-world applications** where appropriate
- Avoid phrases like "According to the text..."
- Test **conceptual understanding** and **critical thinking**

## Rubric Requirements
- Create **3-5 distinct criteria** that don't overlap
- Each criterion assesses **different competencies**
- Include **specific performance descriptors**
- **Total marks must equal exactly 10**
- Use **clear, measurable language**

## Output Format
```
Question X Start
Question: [Analytical question requiring critical thinking]
Rubric:
- [Criterion 1: Specific skill/knowledge] - [X marks] 
- [Criterion 2: Different competency] - [X marks]
- [Criterion 3: Another distinct skill] - [X marks]
- [Additional criteria as needed] - [X marks]
Question X End
```

## Quality Checklist
✓ Questions require analysis/synthesis/evaluation
✓ Rubric criteria are independent and specific
✓ Total marks = 10 for each question
✓ Language is clear and professional
✓ Assesses understanding, not memorization

## Common Rubric Criteria Examples
- **Conceptual Understanding** - Demonstrates grasp of key principles
- **Critical Analysis** - Evaluates evidence and arguments thoughtfully  
- **Application** - Applies knowledge to new situations effectively
- **Communication** - Presents ideas clearly with appropriate terminology
- **Problem-Solving** - Develops logical solutions or approaches

---
**Ready to generate questions. Provide your context, topic, quantity, and academic level.**
context:{context}
topic:{question}
quantity:{no_of_questions}
academic level: undergraduate                                                    
"""
)

    inputs = {"context": context_text, "question": query_text,"no_of_questions":no_of_questions}

    # Run LLM chain with Together.ai
    llm = ChatGoogleGenerativeAI(
    model=model,  # Or "gemini-flash" for the latest flash model
    temperature=temp,
    google_api_key= googleapikey # Uncomment and replace if not using env var
)
    chain = LLMChain(llm=llm, prompt=prompt_template, output_parser=StrOutputParser())
    response = chain.run(inputs)
    print(response)
    output_path+=f"{subject}_Chapter{chapter}_{query_text}.json"
    parse_questions(response,no_of_questions,output_path)


In [9]:
#query_rag("Topic Name",no of question,"directory to save the json","Subject",chapter no)
query_rag("Nature of software",10,"","SoftwareEngineering",1)


C:\Users\dhili\AppData\Local\Temp\ipykernel_11164\4175683258.py:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(
C:\Users\dhili\AppData\Local\Temp\ipykernel_11164\4175683258.py:39: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=vector_db_path, embedding_function=embedding_function)
C:\Users\dhili\AppData\L

**Question 1 Start**

**Question:**  Compare and contrast the characteristics of embedded software, product-line software, web applications, and artificial intelligence software, providing specific examples from the provided text and illustrating how their development processes might differ.  Consider the scale of development effort and the types of challenges each category presents.

**Rubric:**
- **Conceptual Understanding of Software Categories:**  Correctly identifies and differentiates the four software categories (2 marks)
- **Comparative Analysis:**  Provides a clear comparison and contrast of the characteristics, highlighting key similarities and differences (3 marks)
- **Application of Concepts:**  Applies the understanding to explain how development processes might vary based on the nature of each software type (3 marks)
- **Clarity and Organization:** Presents the comparison in a clear, logical, and well-organized manner (2 marks)

**Question 1 End**


**Question 2 Start**

